# Notebook Colab: Detección y Clasificación Bicicleta vs Auto (YOLOv8)

**Autor:** Generado automáticamente.

Este cuaderno implementa un flujo completo para entrenar, evaluar e inferir un modelo YOLOv8 que detecta bicicletas y autos usando datasets en formato YOLO v8 dentro de `Z_ProyectoYolo/datasets`.

---
## Índice de Secciones
0. Setup
1. Definición y justificación
2. Verificación de dataset
3. Visualización de datos y comprobaciones
4. Preparación de `data.yaml`
5. Selección del modelo y justificación
6. Entrenamiento
7. Visualización de logs y métricas
8. Evaluación cuantitativa
9. Inferencia (solo imágenes)
10. Empaquetado y entrega
11. Informe final (markdown y opcional PDF)
12. Checklist final

Ejecute cada sección en orden. Todo el contenido está en español y las rutas son relativas para reproducibilidad en Google Colab.


## 0. Setup

Instala dependencias, detecta Colab y prepara variables y carpetas del proyecto. Las rutas son relativas a `PROJECT_ROOT = 'Z_ProyectoYolo'`.

- Dependencias: `ultralytics`, `opencv-python-headless`, `matplotlib`, `pandas`, `tqdm`, `scikit-learn`, `seaborn`, `pyyaml`.
- Opción de montar Google Drive en Colab (desactivado por defecto; active `DO_MOUNT_DRIVE=True` si desea guardar en Drive).
- Fijación de semillas para reproducibilidad.
- Selección automática de dispositivo.
- Creación de carpetas de salida si no existen.
- Muestra versiones de `ultralytics` y `torch`.


In [ ]:
# Instalación de dependencias y configuración inicial
# Nota: En Colab, esta celda instalará/actualizará las dependencias necesarias.

import os, sys, json, random, shutil, time, glob
from pathlib import Path

# Detectar si estamos en Google Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# (Opcional) Montar Drive en Colab
DO_MOUNT_DRIVE = False  # Cambie a True si desea montar su Google Drive
if IN_COLAB and DO_MOUNT_DRIVE:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)

# Instalar dependencias clave
if IN_COLAB:
    # En Colab use pip mágico para salida limpia
    %pip -q install ultralytics opencv-python-headless matplotlib pandas tqdm scikit-learn seaborn pyyaml
else:
    # En entornos locales también podemos instalar si faltan
    import subprocess
    def _pip_install(pkgs):
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', *pkgs])
        except Exception as e:
            print('Aviso: no se pudo instalar', pkgs, e)
    _pip_install(['ultralytics','opencv-python-headless','matplotlib','pandas','tqdm','scikit-learn','seaborn','pyyaml'])

import yaml
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# Fijar semillas para reproducibilidad
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
try:
    import torch
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
except Exception:
    pass

# Seleccionar dispositivo
try:
    import torch
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
except Exception:
    DEVICE = 'cpu'

# Definir raíces del proyecto (evitar rutas absolutas fijas; preferir variables)
PROJECT_ROOT = 'Z_ProyectoYolo'
# Si estamos en Colab y el proyecto existe en la ruta de trabajo actual, usarla.
# Si montó Drive y la carpeta existe allí, puede definir manualmente PROJECT_ROOT a esa ubicación.
if IN_COLAB:
    # Preferir carpeta en el cwd; si no existe pero existe en Drive, usarla.
    if not Path(PROJECT_ROOT).exists() and Path('/content/drive/MyDrive/Z_ProyectoYolo').exists():
        PROJECT_ROOT = '/content/drive/MyDrive/Z_ProyectoYolo'

DATASET_ROOT = os.path.join(PROJECT_ROOT, 'datasets')

# Directorios de salida
RUNS_DIR = os.path.join(PROJECT_ROOT, 'runs')
EVAL_DIR = os.path.join(PROJECT_ROOT, 'evaluation')
EVAL_METRICS_DIR = os.path.join(EVAL_DIR, 'metrics')
EVAL_FIG_DIR = os.path.join(EVAL_DIR, 'figures')
INFER_DIR = os.path.join(PROJECT_ROOT, 'inference')
INFER_RESULTS_DIR = os.path.join(INFER_DIR, 'results_images')
INFER_NEW_IMG_DIR = os.path.join(INFER_DIR, 'images_new')
ENTREGA_DIR = os.path.join(PROJECT_ROOT, 'entrega_final')
ENTREGA_MODEL_DIR = os.path.join(ENTREGA_DIR, 'modelo')
DOCS_DIR = os.path.join(PROJECT_ROOT, 'docs')

# Crear carpetas requeridas
for d in [RUNS_DIR, EVAL_METRICS_DIR, EVAL_FIG_DIR, INFER_RESULTS_DIR, INFER_NEW_IMG_DIR, ENTREGA_MODEL_DIR, DOCS_DIR]:
    Path(d).mkdir(parents=True, exist_ok=True)

# Mostrar versiones
from importlib.metadata import version, PackageNotFoundError

def _ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return 'no-instalado'

ultra_ver = _ver('ultralytics')
torch_ver = _ver('torch')
print(f"IN_COLAB={IN_COLAB} | DEVICE={DEVICE}")
print(f"ultralytics=={ultra_ver} | torch=={torch_ver}")

# Guardar requirements.txt con versiones actuales (para reproducibilidad en Colab)
# Nota: esto sobrescribe el requirements.txt con versiones concretas del entorno actual
req_path = os.path.join(PROJECT_ROOT, 'requirements.txt')
try:
    import pkgutil
    import subprocess
    freeze = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'], text=True)
    with open(req_path, 'w') as f:
        f.write(freeze)
    print(f"Guardado {req_path} con versiones instaladas.")
except Exception as e:
    print('Aviso: no se pudo escribir requirements.txt:', e)


## 1. Definición y justificación

**Objetivo:** Entrenar y evaluar un detector YOLOv8 capaz de detectar y clasificar objetos de las clases bicicleta y auto en imágenes.

**Justificación (usabilidad):**
- Conteo de tráfico y análisis de movilidad urbana.
- Seguridad vial y monitoreo de zonas con alta circulación.
- Gestión de estacionamientos y control de acceso en recintos.
- Apoyo a ciudades inteligentes para planificación de infraestructura.


## 2. Verificación de dataset

Validación de estructura, conteo de imágenes por split/clase y detección de inconsistencias. Se generan resúmenes en `evaluation/metrics/` y notas en `docs/labeling_notes.md`.


In [ ]:
# Verificación de la estructura del dataset y conteos
from collections import Counter, defaultdict
import pandas as pd

required_dirs = [
    os.path.join(DATASET_ROOT, 'images', 'train'),
    os.path.join(DATASET_ROOT, 'images', 'val'),
    os.path.join(DATASET_ROOT, 'labels', 'train'),
    os.path.join(DATASET_ROOT, 'labels', 'val'),
]

missing_reqs = [d for d in required_dirs if not Path(d).exists()]
if missing_reqs:
    print("ADVERTENCIA: Faltan directorios requeridos:")
    for d in missing_reqs:
        print(" -", d)
else:
    print("Estructura mínima encontrada.")

IMG_EXT = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp'}

split_stats = {}
label_issues = { 'missing_label_for_image': [], 'empty_label_file': [], 'malformed_lines': [] }
class_counts = { 'train': Counter(), 'val': Counter() }
image_counts = { 'train': 0, 'val': 0 }
instance_counts = { 'train': 0, 'val': 0 }

for split in ['train','val']:
    img_dir = os.path.join(DATASET_ROOT, 'images', split)
    lab_dir = os.path.join(DATASET_ROOT, 'labels', split)
    imgs = []
    if Path(img_dir).exists():
        for p in Path(img_dir).glob('*'):
            if p.suffix.lower() in IMG_EXT:
                imgs.append(p)
    image_counts[split] = len(imgs)

    # Verificar que cada imagen tenga su .txt correspondiente
    for img_path in imgs:
        label_path = Path(lab_dir) / (img_path.stem + '.txt')
        if not label_path.exists():
            label_issues['missing_label_for_image'].append(str(img_path))
            continue
        # Leer archivo de etiquetas
        try:
            with open(label_path, 'r') as f:
                lines = [ln.strip() for ln in f.readlines() if ln.strip()]
        except Exception as e:
            label_issues['malformed_lines'].append(f"{label_path} :: error apertura {e}")
            continue
        if len(lines) == 0:
            label_issues['empty_label_file'].append(str(label_path))
        for ln in lines:
            parts = ln.split()
            if len(parts) != 5:
                label_issues['malformed_lines'].append(f"{label_path} :: '{ln}'")
                continue
            try:
                cls_id = int(float(parts[0]))
                # YOLOv8 usa: class cx cy w h (normalizados)
                _ = [float(x) for x in parts[1:]]
                if cls_id not in [0,1]:
                    label_issues['malformed_lines'].append(f"{label_path} :: clase inválida {cls_id}")
                    continue
                class_counts[split][cls_id] += 1
                instance_counts[split] += 1
            except Exception:
                label_issues['malformed_lines'].append(f"{label_path} :: '{ln}' (parse)")

# Resumen por split
summary_rows = []
for split in ['train','val']:
    row = {
        'split': split,
        'images': image_counts[split],
        'instances_total': instance_counts[split],
        'instances_bicicleta(cls0)': class_counts[split][0],
        'instances_auto(cls1)': class_counts[split][1],
    }
    summary_rows.append(row)
summary_df = pd.DataFrame(summary_rows)
summary_df

# Guardar resumen
summary_csv = os.path.join(EVAL_METRICS_DIR, 'dataset_summary.csv')
summary_json = os.path.join(EVAL_METRICS_DIR, 'dataset_summary.json')
summary_df.to_csv(summary_csv, index=False)
with open(summary_json, 'w') as f:
    json.dump({
        'images': image_counts,
        'instances': instance_counts,
        'class_counts': {k: dict(v) for k,v in class_counts.items()},
        'issues_counts': {k: len(v) for k,v in label_issues.items()}
    }, f, indent=2)
print('Guardado resumen en:', summary_csv, 'y', summary_json)

# Guardar notas de etiquetado
label_notes = os.path.join(DOCS_DIR, 'labeling_notes.md')
with open(label_notes, 'a') as f:
    f.write('\n\n## Verificación automática\n')
    f.write(f"\nFecha: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
    if label_issues['missing_label_for_image']:
        f.write('\n### Imágenes sin archivo de etiqueta (.txt)\n')
        for p in label_issues['missing_label_for_image']:
            f.write(f"- {p}\n")
    if label_issues['empty_label_file']:
        f.write('\n### Archivos de etiqueta vacíos\n')
        for p in label_issues['empty_label_file']:
            f.write(f"- {p}\n")
    if label_issues['malformed_lines']:
        f.write('\n### Líneas mal formateadas\n')
        for desc in label_issues['malformed_lines']:
            f.write(f"- {desc}\n")

# Advertencia por desbalance o escasez (<50 instancias por clase)
for split in ['train','val']:
    for cls_id, cls_name in [(0,'bicicleta'),(1,'auto')]:
        if class_counts[split][cls_id] < 50:
            print(f"ADVERTENCIA: Clase '{cls_name}' en split '{split}' tiene menos de 50 instancias ({class_counts[split][cls_id]}). Considere aumentar los datos.")

# Mensaje de balance
def _imbalance_msg(cc):
    total = cc[0] + cc[1]
    if total == 0:
        return 'sin instancias'
    ratio = (cc[0] / total) if total > 0 else 0
    return f"bicicleta={cc[0]} ({ratio:.1%}), auto={cc[1]} ({1-ratio:.1%})"

print('Distribución train:', _imbalance_msg(class_counts['train']))
print('Distribución val  :', _imbalance_msg(class_counts['val']))


## 3. Visualización de datos y comprobaciones

Se muestran 8–12 ejemplos aleatorios del split de entrenamiento con bounding boxes dibujados a partir de los archivos de etiqueta YOLO (`class cx cy w h`). La figura se guarda en `evaluation/figures/sample_grid.png`.


In [ ]:
# Visualización aleatoria de ejemplos etiquetados del split train
import math

train_img_dir = os.path.join(DATASET_ROOT, 'images', 'train')
train_lab_dir = os.path.join(DATASET_ROOT, 'labels', 'train')
all_train_imgs = [p for p in Path(train_img_dir).glob('*') if p.suffix.lower() in IMG_EXT]

n_show = 12 if len(all_train_imgs) >= 12 else max(1, len(all_train_imgs))
sel = random.sample(all_train_imgs, n_show) if len(all_train_imgs) >= n_show else all_train_imgs

class_names = {0: 'bicicleta', 1: 'auto'}
fig_cols = 4
fig_rows = math.ceil(len(sel)/fig_cols)
plt.figure(figsize=(16, 4*fig_rows))

missing_label_imgs = []

for i, img_path in enumerate(sel, 1):
    lab_path = Path(train_lab_dir) / (img_path.stem + '.txt')
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]
    if not lab_path.exists():
        missing_label_imgs.append(str(img_path))
        boxes = []
    else:
        with open(lab_path, 'r') as f:
            lines = [ln.strip() for ln in f.readlines() if ln.strip()]
        boxes = []
        for ln in lines:
            parts = ln.split()
            if len(parts) != 5:
                continue
            try:
                cls_id = int(float(parts[0]))
                cx, cy, w, h = map(float, parts[1:])
                # Convertir normalizados a pixeles
                x1 = int((cx - w/2) * W)
                y1 = int((cy - h/2) * H)
                x2 = int((cx + w/2) * W)
                y2 = int((cy + h/2) * H)
                boxes.append((x1,y1,x2,y2,cls_id))
            except Exception:
                continue
    # Dibujar
    img_draw = img.copy()
    for (x1,y1,x2,y2,cls_id) in boxes:
        color = (0,255,0) if cls_id == 1 else (255,0,0)
        cv2.rectangle(img_draw, (x1,y1), (x2,y2), color, 2)
        label = class_names.get(cls_id, f"cls{cls_id}")
        cv2.putText(img_draw, label, (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    plt.subplot(fig_rows, fig_cols, i)
    plt.imshow(img_draw)
    plt.axis('off')
    plt.title(img_path.name)

plt.tight_layout()
fig_path = os.path.join(EVAL_FIG_DIR, 'sample_grid.png')
plt.savefig(fig_path, dpi=150)
plt.show()
print('Guardada figura de ejemplos en:', fig_path)

# Registrar imágenes sin label en labeling_notes.md
if missing_label_imgs:
    with open(os.path.join(DOCS_DIR, 'labeling_notes.md'), 'a') as f:
        f.write('\n### Imágenes sin label detectadas en visualización (train)\n')
        for p in missing_label_imgs:
            f.write(f"- {p}\n")


## 4. Preparación de `data.yaml`

Se crea y valida el archivo de configuración con rutas relativas y nombres de clases.


In [ ]:
# Crear/validar data.yaml con rutas relativas y clases
DATA_YAML_PATH = os.path.join(PROJECT_ROOT, 'data.yaml')

data_cfg = {
    'train': 'Z_ProyectoYolo/datasets/images/train',
    'val':   'Z_ProyectoYolo/datasets/images/val',
    'nc': 2,
    'names': ['bicicleta','auto']
}

with open(DATA_YAML_PATH, 'w') as f:
    yaml.safe_dump(data_cfg, f, sort_keys=False, allow_unicode=True)

print('Escrito:', DATA_YAML_PATH)
print('Contenido:')
print(yaml.safe_dump(data_cfg, sort_keys=False, allow_unicode=True))

# Validar rutas
assert Path(os.path.join(PROJECT_ROOT, 'datasets', 'images', 'train')).exists(), 'No existe images/train'
assert Path(os.path.join(PROJECT_ROOT, 'datasets', 'images', 'val')).exists(), 'No existe images/val'
assert Path(os.path.join(PROJECT_ROOT, 'datasets', 'labels', 'train')).exists(), 'No existe labels/train'
assert Path(os.path.join(PROJECT_ROOT, 'datasets', 'labels', 'val')).exists(), 'No existe labels/val'
print('Validación básica de rutas OK.')


## 5. Selección del modelo y justificación

Se utiliza `yolov8n.pt` por su ligereza y buena compatibilidad con Colab gratuito. Alternativa: `yolov8s.pt` para mayor precisión si la GPU lo permite. Hiperparámetros iniciales:
- `IMGSZ = 640`
- `EPOCHS = 50`
- `BATCH = 16` (si OOM: intentar 8, luego 4)
- Augmentaciones por defecto de YOLOv8: mosaic, volteo horizontal, HSV, escala.


In [ ]:
# Definición de hiperparámetros y selección de modelo
MODEL_BACKBONE = 'yolov8n.pt'  # alternativa: 'yolov8s.pt'
IMGSZ = 640
EPOCHS = 50
BATCH = 16
print(f"Modelo: {MODEL_BACKBONE} | IMGSZ={IMGSZ} | EPOCHS={EPOCHS} | BATCH={BATCH}")


## 6. Entrenamiento

Comando reproducible (CLI):

```
!yolo detect train model={MODEL_BACKBONE} data=Z_ProyectoYolo/data.yaml imgsz=640 epochs=50 batch=16 project=runs name=exp_bici_auto
```

A continuación se ejecuta el entrenamiento con la API de Python (manejo de OOM reduciendo batch 16→8→4). Se normaliza la ubicación de `best.pt` a `Z_ProyectoYolo/runs/exp_bici_auto/weights/best.pt` si Ultralytics usa subcarpeta `detect/`.


In [ ]:
# Entrenamiento con reintento ante OOM y normalización de rutas de salida
from ultralytics import YOLO

os.makedirs(RUNS_DIR, exist_ok=True)

# Asegurar cwd a raíz del proyecto (para que runs/ quede dentro de Z_ProyectoYolo)
os.chdir(Path(PROJECT_ROOT))
print('CWD for training:', os.getcwd())

cli_cmd = f"yolo detect train model={MODEL_BACKBONE} data=Z_ProyectoYolo/data.yaml imgsz={IMGSZ} epochs={EPOCHS} batch={BATCH} project=runs name=exp_bici_auto"
print('Comando reproducible CLI:\n', cli_cmd)

batches = [BATCH, 8, 4] if BATCH != 8 else [8, 4]
train_success = False
train_save_dir = None
for b in batches:
    try:
        print(f"\nIntentando entrenamiento con batch={b} ...")
        model = YOLO(MODEL_BACKBONE)
        results = model.train(
            data='Z_ProyectoYolo/data.yaml',
            imgsz=IMGSZ,
            epochs=EPOCHS,
            batch=b,
            project='runs',
            name='exp_bici_auto',
            verbose=True,
        )
        # Intentar obtener el directorio de guardado
        try:
            train_save_dir = str(model.trainer.save_dir)
        except Exception:
            train_save_dir = None
        train_success = True
        break
    except RuntimeError as e:
        if 'out of memory' in str(e).lower():
            print('OOM detectado. Probando con batch menor...')
            continue
        else:
            raise e

if not train_success:
    raise RuntimeError('Entrenamiento falló en todos los tamaños de batch.')

# Normalizar ruta: asegurar Z_ProyectoYolo/runs/exp_bici_auto/weights/best.pt
norm_run_dir = os.path.join('Z_ProyectoYolo', 'runs', 'exp_bici_auto')
Path(norm_run_dir).mkdir(parents=True, exist_ok=True)

# Buscar best.pt en subcarpetas de runs
best_candidate = None
for p in Path('Z_ProyectoYolo/runs').rglob('best.pt'):
    if 'exp_bici_auto' in str(p):
        best_candidate = p
        break

if best_candidate is None:
    # También revisar si Ultralytics guardó en runs/exp_bici_auto directamente (con cwd en PROJECT_ROOT)
    for p in Path('runs').rglob('best.pt'):
        if 'exp_bici_auto' in str(p):
            best_candidate = p
            break

if best_candidate is None:
    print('ADVERTENCIA: No se encontró best.pt. Verifique el entrenamiento.')
else:
    dest_dir = os.path.join('Z_ProyectoYolo', 'runs', 'exp_bici_auto', 'weights')
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    dest_path = os.path.join(dest_dir, 'best.pt')
    shutil.copy2(str(best_candidate), dest_path)
    print('best.pt normalizado a:', dest_path)

# Volver cwd por seguridad
os.chdir('..')


## 7. Visualización de logs y métricas

Se extraen métricas de entrenamiento (losses, precisión, recall, mAP) y se generan gráficos guardados en `evaluation/figures/`.


In [ ]:
# Cargar logs de entrenamiento y graficar métricas
import pandas as pd

# Intentar localizar el directorio del experimento
candidates = [
    os.path.join(PROJECT_ROOT, 'runs', 'exp_bici_auto'),
    os.path.join(PROJECT_ROOT, 'runs', 'detect', 'exp_bici_auto'),
]
run_dir = None
for c in candidates:
    if Path(c).exists():
        run_dir = c
        break

if run_dir is None:
    # Buscar por results.csv en runs
    for p in Path(os.path.join(PROJECT_ROOT, 'runs')).rglob('results.csv'):
        if 'exp_bici_auto' in str(p):
            run_dir = str(p.parent)
            break

if run_dir is None:
    raise FileNotFoundError('No se encontró el directorio de corrida con results.csv')

results_csv = os.path.join(run_dir, 'results.csv')
print('results.csv:', results_csv)
results = pd.read_csv(results_csv)

# Graficar pérdidas por epoch
loss_cols = [c for c in results.columns if 'loss' in c.lower()]
plt.figure(figsize=(10,6))
for col in loss_cols:
    plt.plot(results.index, results[col], label=col)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Pérdidas por epoch')
plt.legend()
loss_fig = os.path.join(EVAL_FIG_DIR, 'losses.png')
plt.savefig(loss_fig, dpi=150)
plt.show()
print('Guardado gráfico de pérdidas en:', loss_fig)

# Graficar Precision/Recall y mAP
metric_cols = {}
for key in ['precision', 'recall', 'mAP50', 'mAP50-95']:
    # Buscar columnas que contengan las claves típicas de Ultralytics
    matches = [c for c in results.columns if key.lower() in c.lower()]
    if matches:
        metric_cols[key] = matches[0]

plt.figure(figsize=(10,6))
for key, col in metric_cols.items():
    plt.plot(results.index, results[col], label=key)
plt.xlabel('epoch')
plt.ylabel('valor')
plt.title('Precision / Recall / mAP')
plt.legend()
pr_fig = os.path.join(EVAL_FIG_DIR, 'pr_map.png')
plt.savefig(pr_fig, dpi=150)
plt.show()
print('Guardado gráfico PR/mAP en:', pr_fig)

# Consolidar métricas finales
final_metrics = {}
for key, col in metric_cols.items():
    final_metrics[key] = float(results[col].iloc[-1])

# Guardar en metrics.json (merge con dataset_summary)
metrics_json = os.path.join(EVAL_METRICS_DIR, 'metrics.json')
base = {}
if Path(metrics_json).exists():
    try:
        base = json.load(open(metrics_json, 'r'))
    except Exception:
        base = {}
base.update({'training': {
    'run_dir': run_dir,
    'final_metrics': final_metrics,
}})
with open(metrics_json, 'w') as f:
    json.dump(base, f, indent=2)
print('Métricas consolidadas en:', metrics_json)


## 8. Evaluación cuantitativa

Se ejecuta validación con el mejor modelo y se construye la matriz de confusión, consolidando métricas (Precision, Recall, mAP@0.5, mAP@0.5:0.95, IoU promedio). Fórmula del IoU:

$$IoU = \frac{|A \cap B|}{|A \cup B|}$$


In [ ]:
# Validación y matriz de confusión
from ultralytics import YOLO

best_path = os.path.join(PROJECT_ROOT, 'runs', 'exp_bici_auto', 'weights', 'best.pt')
if not Path(best_path).exists():
    # Buscar fallback
    for p in Path(os.path.join(PROJECT_ROOT, 'runs')).rglob('best.pt'):
        if 'exp_bici_auto' in str(p):
            best_path = str(p)
            break
print('Usando modelo para evaluación:', best_path)

model = YOLO(best_path)
val_res = model.val(data=os.path.join(PROJECT_ROOT, 'data.yaml'), imgsz=IMGSZ)

# Extraer métricas
# Ultralytics produce un dict-like accesible a través de val_res.results_dict (en versiones recientes)
metrics_out = {}
try:
    metrics_out = dict(val_res.results_dict)
except Exception:
    # Construir manualmente si no está disponible
    try:
        metrics_out = {
            'precision': float(val_res.box.maps[0]) if hasattr(val_res, 'box') and hasattr(val_res.box, 'maps') else None
        }
    except Exception:
        metrics_out = {}

# Intentar recuperar PR/mAP de archivos
res_csv = None
for p in Path(os.path.join(PROJECT_ROOT, 'runs')).rglob('results.csv'):
    if 'exp_bici_auto' in str(p):
        res_csv = p
        break
if res_csv is not None:
    df = pd.read_csv(res_csv)
    def _pick(colname):
        cand = [c for c in df.columns if colname.lower() in c.lower()]
        return float(df[cand[0]].iloc[-1]) if cand else None
    metrics_out.update({
        'precision': metrics_out.get('precision') or _pick('precision'),
        'recall': _pick('recall'),
        'mAP50': _pick('map50'),
        'mAP50-95': _pick('map50-95'),
    })

# IoU promedio (si disponible)
try:
    if hasattr(val_res, 'box') and hasattr(val_res.box, 'iou'):  # versiones futuras
        metrics_out['IoU'] = float(np.mean(val_res.box.iou))
except Exception:
    pass

# Guardar matriz de confusión si existe
cm_png = None
for p in Path(os.path.join(PROJECT_ROOT, 'runs')).rglob('confusion_matrix.png'):
    if 'exp_bici_auto' in str(p):
        cm_png = str(p)
        break
if cm_png:
    # Copiar a evaluation/figures
    dest_cm = os.path.join(EVAL_FIG_DIR, 'confusion_matrix.png')
    shutil.copy2(cm_png, dest_cm)
    print('Matriz de confusión guardada en:', dest_cm)

# Guardar métricas
metrics_json = os.path.join(EVAL_METRICS_DIR, 'metrics.json')
base = {}
if Path(metrics_json).exists():
    try:
        base = json.load(open(metrics_json, 'r'))
    except Exception:
        base = {}
base['validation'] = metrics_out
with open(metrics_json, 'w') as f:
    json.dump(base, f, indent=2)
print('Métricas de evaluación guardadas en:', metrics_json)

# Sugerencias automáticas
sugs = []
if metrics_out.get('mAP50', 0) and metrics_out['mAP50'] < 0.6:
    sugs.append('Recolectar más datos y balancear clases (especialmente la minoritaria).')
    sugs.append('Aumentar épocas o probar backbone más grande (yolov8s).')
if metrics_out.get('precision', 1) < 0.6:
    sugs.append('Revisar anotaciones (falsos positivos pueden indicar cajas imprecisas).')
if metrics_out.get('recall', 1) < 0.6:
    sugs.append('Mejorar cobertura de datos y diversidad; ajustar umbral de confianza en inferencia.')
print('Sugerencias:', '\n- ' + '\n- '.join(sugs) if sugs else 'Modelo con desempeño razonable.')


## 9. Inferencia (solo imágenes)

Se ejecuta inferencia sobre `datasets/images/val` y, si existen, sobre `inference/images_new/`. Las imágenes con predicciones se consolidan en `inference/results_images/`.


In [ ]:
# Inferencia con best.pt sobre val y (opcional) imágenes nuevas
from ultralytics import YOLO

best_path = os.path.join(PROJECT_ROOT, 'runs', 'exp_bici_auto', 'weights', 'best.pt')
assert Path(best_path).exists(), f"No se encontró el modelo: {best_path}"

model = YOLO(best_path)

# Función para predecir y consolidar imágenes de salida
def run_predict_and_collect(source_dir, target_dir, imgsz=IMGSZ, conf=0.25):
    if not Path(source_dir).exists():
        print('No existe fuente de imágenes:', source_dir)
        return []
    print('Inferencia en:', source_dir)
    res = model.predict(source=source_dir, imgsz=imgsz, conf=conf, save=True, project=os.path.join(PROJECT_ROOT, 'inference'), name='tmp_pred', exist_ok=True)
    # Buscar imágenes resultantes y copiarlas a target_dir
    Path(target_dir).mkdir(parents=True, exist_ok=True)
    saved_imgs = []
    # La salida suele estar en PROJECT_ROOT/inference/tmp_pred/predictX
    out_base = os.path.join(PROJECT_ROOT, 'inference', 'tmp_pred')
    candidate = None
    # Preferir último directorio predict*
    preds = sorted([p for p in Path(out_base).glob('predict*') if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    if preds:
        candidate = preds[0]
    if candidate:
        for imgp in candidate.glob('*.jpg'):
            dest = Path(target_dir) / imgp.name
            shutil.copy2(str(imgp), str(dest))
            saved_imgs.append(str(dest))
        for imgp in candidate.glob('*.png'):
            dest = Path(target_dir) / imgp.name
            shutil.copy2(str(imgp), str(dest))
            saved_imgs.append(str(dest))
    print(f"{len(saved_imgs)} imágenes guardadas en {target_dir}")
    return saved_imgs

# Inferencia en val
val_img_dir = os.path.join(PROJECT_ROOT, 'datasets', 'images', 'val')
val_preds = run_predict_and_collect(val_img_dir, INFER_RESULTS_DIR)

# Inferencia en nuevas imágenes (si las hay)
new_preds = run_predict_and_collect(INFER_NEW_IMG_DIR, INFER_RESULTS_DIR)

# Mostrar algunos aciertos/fallos
def show_examples(paths, title, max_n=6):
    if not paths:
        print('Sin ejemplos para', title)
        return
    n = min(max_n, len(paths))
    sel = random.sample(paths, n)
    cols = 3
    rows = math.ceil(n/cols)
    plt.figure(figsize=(12, 4*rows))
    for i,p in enumerate(sel,1):
        img = cv2.imread(p)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img is not None else None
        plt.subplot(rows, cols, i)
        if img is None:
            plt.text(0.5,0.5,'No se pudo cargar', ha='center', va='center')
        else:
            plt.imshow(img); plt.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    out_name = 'qualitative_examples.png' if 'aciertos' in title.lower() else 'failure_cases.png'
    out_path = os.path.join(EVAL_FIG_DIR, out_name)
    plt.savefig(out_path, dpi=150)
    plt.show()
    print('Guardado en:', out_path)

show_examples(val_preds[:], 'Ejemplos de inferencia en val (aciertos y fallos mixtos)')


## 10. Empaquetado y entrega

Se empaquetan artefactos a `Z_ProyectoYolo/entrega_final/entrega_final.zip` y se imprimen los comandos para descargar en Colab.


In [ ]:
# Empaquetar artefactos finales

# Guardar el propio cuaderno dentro del proyecto con nombre canónico
try:
    from IPython.display import display, Javascript
    # Nota: En Colab, guardar el notebook requiere acción del usuario
    print('Asegúrese de guardar este notebook como Z_ProyectoYolo/notebook.ipynb')
except Exception:
    pass

# Copiar artefactos a entrega_final/
NB_PATH = os.path.join(PROJECT_ROOT, 'notebook.ipynb')
REQ_PATH = os.path.join(PROJECT_ROOT, 'requirements.txt')
DATA_YAML_PATH = os.path.join(PROJECT_ROOT, 'data.yaml')
BEST_PT = os.path.join(PROJECT_ROOT, 'runs', 'exp_bici_auto', 'weights', 'best.pt')

os.makedirs(ENTREGA_MODEL_DIR, exist_ok=True)
if Path(BEST_PT).exists():
    shutil.copy2(BEST_PT, os.path.join(ENTREGA_MODEL_DIR, 'best.pt'))

# Copiar carpetas seleccionadas
def copy_tree(src, dst):
    if not Path(src).exists():
        return
    Path(dst).mkdir(parents=True, exist_ok=True)
    for root, dirs, files in os.walk(src):
        rel = os.path.relpath(root, src)
        out_dir = os.path.join(dst, rel)
        Path(out_dir).mkdir(parents=True, exist_ok=True)
        for fn in files:
            shutil.copy2(os.path.join(root, fn), os.path.join(out_dir, fn))

copy_tree(os.path.join(PROJECT_ROOT, 'evaluation'), os.path.join(ENTREGA_DIR, 'evaluation'))
copy_tree(os.path.join(PROJECT_ROOT, 'inference'), os.path.join(ENTREGA_DIR, 'inference'))
copy_tree(os.path.join(PROJECT_ROOT, 'docs'), os.path.join(ENTREGA_DIR, 'docs'))

# Copiar archivos sueltos
for p in [NB_PATH, REQ_PATH, DATA_YAML_PATH, os.path.join(PROJECT_ROOT, 'informe.md')]:
    if Path(p).exists():
        shutil.copy2(p, ENTREGA_DIR)

# Crear ZIP
zip_base = os.path.join(PROJECT_ROOT, 'entrega_final', 'entrega_final')
zip_path = shutil.make_archive(zip_base, 'zip', ENTREGA_DIR)
print('ZIP creado en:', zip_path)

# Comando para descargar en Colab
print('\nPara descargar en Colab ejecute:')
print("from google.colab import files; files.download('Z_ProyectoYolo/entrega_final/entrega_final.zip')")


## 11. Informe final (3–5 páginas en `informe.md`)

Se auto-genera un informe en Markdown con las secciones pedidas. Opcionalmente, se puede convertir a PDF si `pandoc` está disponible en el entorno.


In [ ]:
# Generación automática de informe en markdown

def _read_metrics(metrics_json):
    if Path(metrics_json).exists():
        try:
            return json.load(open(metrics_json,'r'))
        except Exception:
            return {}
    return {}

def _list_images(dir_path, exts={'.png','.jpg','.jpeg'}):
    if not Path(dir_path).exists():
        return []
    return [str(p) for p in Path(dir_path).glob('*') if p.suffix.lower() in exts]

metrics_json = os.path.join(EVAL_METRICS_DIR, 'metrics.json')
metrics_all = _read_metrics(metrics_json)

# Selección de imágenes ejemplo
qual_imgs = _list_images(EVAL_FIG_DIR)
inf_imgs = _list_images(INFER_RESULTS_DIR)

md_lines = []
md_lines.append('# Informe del Proyecto: Detección Bicicleta vs Auto (YOLOv8)')
md_lines.append('')
md_lines.append('## 1. Problema y justificación (usabilidad)')
md_lines.append('- Detección y clasificación de bicicletas y autos en imágenes fijas.')
md_lines.append('- Casos de uso: conteo de tráfico, seguridad vial, estacionamientos, ciudades inteligentes.')
md_lines.append('')
md_lines.append('## 2. Dataset y estructura')
md_lines.append(f"- Raíz del dataset: `{DATASET_ROOT}`")
md_lines.append('- Estructura: images/{train,val} y labels/{train,val} en formato YOLO.')
md_lines.append('- Verificación automática en `evaluation/metrics/dataset_summary.*`.')
md_lines.append('')
md_lines.append('## 3. Configuración de entrenamiento')
md_lines.append(f"- Modelo: `{MODEL_BACKBONE}` | IMGSZ={IMGSZ} | EPOCHS={EPOCHS}")
md_lines.append('- Aumentos: mosaic, flip horizontal, HSV, scale (por defecto YOLOv8).')
md_lines.append('')
md_lines.append('## 4. Resultados y métricas')
train_metrics = metrics_all.get('training', {}).get('final_metrics', {})
val_metrics = metrics_all.get('validation', {})
md_lines.append('### Métricas de entrenamiento (última época)')
md_lines.append('' if train_metrics else '*No disponible*')
for k,v in train_metrics.items():
    md_lines.append(f"- {k}: {v:.4f}")
md_lines.append('')
md_lines.append('### Métricas de validación')
if val_metrics:
    for k,v in val_metrics.items():
        try:
            md_lines.append(f"- {k}: {float(v):.4f}")
        except Exception:
            md_lines.append(f"- {k}: {v}")
else:
    md_lines.append('*No disponible*')
md_lines.append('')
md_lines.append('Se adjuntan figuras en `evaluation/figures/` y ejemplos cualitativos en `inference/results_images/`.')
md_lines.append('')
md_lines.append('## 5. Conclusiones y recomendaciones')
md_lines.append('- Aumentar datos para la clase minoritaria si hay desbalance.')
md_lines.append('- Mejorar calidad de anotación en casos dudosos.')
md_lines.append('- Probar `yolov8s.pt` y ajustar épocas si la GPU lo permite.')

# Escribir informe
informe_path = os.path.join(PROJECT_ROOT, 'informe.md')
with open(informe_path, 'w') as f:
    f.write('\n'.join(md_lines))
print('Informe generado en:', informe_path)

# Conversión opcional a PDF si pandoc está disponible
try:
    import subprocess
    pdf_out = os.path.join(ENTREGA_DIR, 'informe.pdf')
    subprocess.run(['pandoc', informe_path, '-o', pdf_out], check=False)
    if Path(pdf_out).exists():
        print('Informe PDF guardado en:', pdf_out)
except Exception:
    pass


## 12. Checklist final (imprimible)

Se verifica la existencia de los principales artefactos y rutas requeridas; se imprime una lista con ✅/❌.


In [ ]:
# Checklist final de artefactos
checks = [
    ('Z_ProyectoYolo/datasets/images/train', os.path.join(PROJECT_ROOT, 'datasets', 'images', 'train')),
    ('Z_ProyectoYolo/datasets/images/val', os.path.join(PROJECT_ROOT, 'datasets', 'images', 'val')),
    ('Z_ProyectoYolo/datasets/labels/train', os.path.join(PROJECT_ROOT, 'datasets', 'labels', 'train')),
    ('Z_ProyectoYolo/datasets/labels/val', os.path.join(PROJECT_ROOT, 'datasets', 'labels', 'val')),
    ('Z_ProyectoYolo/data.yaml', os.path.join(PROJECT_ROOT, 'data.yaml')),
    ('Z_ProyectoYolo/runs/exp_bici_auto/weights/best.pt', os.path.join(PROJECT_ROOT, 'runs', 'exp_bici_auto', 'weights', 'best.pt')),
    ('Z_ProyectoYolo/evaluation/metrics.json', os.path.join(PROJECT_ROOT, 'evaluation', 'metrics', 'metrics.json')),
    ('Z_ProyectoYolo/evaluation/figures/', os.path.join(PROJECT_ROOT, 'evaluation', 'figures')),
    ('Z_ProyectoYolo/inference/results_images/', os.path.join(PROJECT_ROOT, 'inference', 'results_images')),
    ('Z_ProyectoYolo/docs/labeling_notes.md', os.path.join(PROJECT_ROOT, 'docs', 'labeling_notes.md')),
    ('Z_ProyectoYolo/informe.md', os.path.join(PROJECT_ROOT, 'informe.md')),
    ('Z_ProyectoYolo/entrega_final/entrega_final.zip', os.path.join(PROJECT_ROOT, 'entrega_final', 'entrega_final.zip')),
]

for label, path in checks:
    exists = Path(path).exists()
    mark = '✅' if exists else '❌'
    print(f"{mark} {label}")

print('\nSi está en Colab y desea descargar el ZIP, ejecute:')
print("from google.colab import files; files.download('Z_ProyectoYolo/entrega_final/entrega_final.zip')")
